# Introduction

Express concatenation using SQL.

Then use that expression to find examples of concatenation.

In [1]:
import duckdb

In [2]:
res = duckdb.sql("""
select * from '../experiments/results/parquet_tall/df_*.parquet'
limit 10
""")
res.show()


┌───────┬───────┬───────┬───────┬───────┐
│ index │  fs   │   i   │   j   │   v   │
│ int64 │ int32 │ int16 │ uint8 │ int32 │
├───────┼───────┼───────┼───────┼───────┤
│     0 │    26 │     0 │     0 │     7 │
│     1 │    26 │     0 │     1 │    14 │
│     2 │    26 │     0 │     2 │     9 │
│     3 │    26 │     0 │     3 │     8 │
│     4 │    26 │     0 │     4 │     2 │
│     5 │    26 │     0 │     5 │    12 │
│     6 │    26 │     0 │     6 │     6 │
│     7 │    26 │     0 │     7 │    10 │
│     8 │    26 │     0 │     8 │     3 │
│     9 │    26 │     0 │     9 │    11 │
├───────┴───────┴───────┴───────┴───────┤
│ 10 rows                     5 columns │
└───────────────────────────────────────┘



In [3]:
res = duckdb.sql("""
select
    fs, count(1)
from '../experiments/results/parquet_tall/df_*.parquet'
group by fs
limit 10
""")
res.show()


┌───────┬──────────┐
│  fs   │ count(1) │
│ int32 │  int64   │
├───────┼──────────┤
│ 61304 │       24 │
│ 61309 │       16 │
│ 61313 │       41 │
│ 61320 │       37 │
│ 61335 │       52 │
│ 29564 │       13 │
│ 29607 │       13 │
│ 29849 │       13 │
│ 29905 │      423 │
│ 29962 │        6 │
├───────┴──────────┤
│     10 rows      │
└──────────────────┘



## Relational algebra
A concatenation is expressed as follows in the RA:

Select topmost cells:
$$
\textsc{TopM}(A)=A\setminus \pi_{i_{2},j_{2},v_{2}}\sigma_{j_{1}>j_{2}}A\times A
$$

Select bottommost cells:
$$
\textsc{BottomM}(A)=A\setminus \pi_{i_{2},j_{2},v_{2}}\sigma_{j_{1}<j_{2}}A\times A
$$

Concatenate vertically:
$$
A-B=A \cup \pi_{i_{3} + (i_{1}-i_{2}), j_{3}, v_{3}}\textsc{Bottom}(A) \times \textsc{TopM}(B) \times B
$$

In [4]:
# Now implement as SQL
res = duckdb.sql("""
with A as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    where fs = 138384
),
B as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    where fs = 138390
),
moved_B as (
    select index, fs, 1 + i + ((select max(i) from A) - (select min(i) from B)) as i, j, v
    from B
)
select * from A
union all
select * from moved_B
limit 100
""")
res.show()


┌────────┬────────┬───────┬───────┬─────────┐
│ index  │   fs   │   i   │   j   │    v    │
│ int64  │ int32  │ int16 │ uint8 │  int32  │
├────────┼────────┼───────┼───────┼─────────┤
│ 113500 │ 138384 │     0 │     0 │  225613 │
│ 113600 │ 138384 │     1 │     0 │  635174 │
│ 113700 │ 138384 │     2 │     0 │ 1169398 │
│ 113900 │ 138384 │     4 │     0 │ 1169728 │
│ 114501 │ 138384 │    10 │     1 │ 1169794 │
│ 114601 │ 138384 │    11 │     1 │   56687 │
│ 114605 │ 138384 │    11 │     5 │    1383 │
│ 114609 │ 138384 │    11 │     9 │     220 │
│ 114700 │ 138384 │    12 │     0 │  581026 │
│ 114701 │ 138384 │    12 │     1 │ 1169795 │
│    ·   │    ·   │     · │     · │     ·   │
│    ·   │    ·   │     · │     · │     ·   │
│    ·   │    ·   │     · │     · │     ·   │
│ 131900 │ 138390 │    39 │     0 │  225988 │
│ 131901 │ 138390 │    39 │     1 │  225993 │
│ 131905 │ 138390 │    39 │     5 │    4861 │
│ 132000 │ 138390 │    40 │     0 │  225992 │
│ 132100 │ 138390 │    41 │     0 

In [5]:
rdf = res.to_df()
rdf

,index,fs,i,j,v
0,113500,138384,0,0,225613
1,113600,138384,1,0,635174
2,113700,138384,2,0,1169398
3,113900,138384,4,0,1169728
4,114501,138384,10,1,1169794
5,114601,138384,11,1,56687
6,114605,138384,11,5,1383
7,114609,138384,11,9,220
8,114700,138384,12,0,581026
9,114701,138384,12,1,1169795


In [6]:
rdf['ov'] = rdf.fs.astype(str) + ' ' + rdf.v.astype(str)
rdf.pivot(index='i', columns='j', values='ov')

j,0,1,5,9
i,,,,
0,138384 225613,NaN,NaN,NaN
1,138384 635174,NaN,NaN,NaN
2,138384 1169398,NaN,NaN,NaN
4,138384 1169728,NaN,NaN,NaN
10,NaN,138384 1169794,NaN,NaN
11,NaN,138384 56687,138384 1383,138384 220
12,138384 581026,138384 1169795,138384 707191,138384 1169793
13,138384 652480,138384 301291,138384 360625,138384 301228
14,138384 225750,138384 284653,138384 139628,138384 548760


## Find common subsets

In [7]:
res = duckdb.sql("""
with A as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    limit 10000
)
SELECT DISTINCT a.fs AS Id1, b.fs AS Id2
FROM
    (SELECT DISTINCT fs FROM A) a
CROSS JOIN
    (SELECT DISTINCT fs FROM A) b
WHERE a.fs < b.fs
  AND EXISTS (
    SELECT v FROM A WHERE fs = a.fs
    INTERSECT
    SELECT v FROM A WHERE fs = b.fs
  );
""")
res

┌───────┬───────┐
│  Id1  │  Id2  │
│ int32 │ int32 │
├───────┼───────┤
│   113 │   314 │
│   136 │   314 │
│   113 │   344 │
│   136 │   344 │
│    94 │   314 │
│    94 │   344 │
│   343 │   346 │
│   245 │   343 │
│   113 │   137 │
│   136 │   137 │
│     · │    ·  │
│     · │    ·  │
│     · │    ·  │
│    94 │   331 │
│    94 │   346 │
│    87 │    92 │
│    57 │    92 │
│   252 │   343 │
│   314 │   345 │
│   344 │   345 │
│    88 │    89 │
│    88 │    94 │
│    89 │    94 │
├───────┴───────┤
│   253 rows    │
│  (20 shown)   │
└───────────────┘

In [8]:
res = duckdb.sql("""
with A as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    limit 10000
)
SELECT DISTINCT a.fs AS Id1, b.fs AS Id2, c.fs as Id3
FROM
    (SELECT DISTINCT fs FROM A) a
CROSS JOIN
    (SELECT DISTINCT fs FROM A) b
CROSS JOIN
    (SELECT DISTINCT fs FROM A) c
WHERE a.fs < b.fs and a.fs < c.fs
  AND EXISTS (
    SELECT v FROM A WHERE fs = a.fs
    INTERSECT
    SELECT v FROM A WHERE fs = b.fs
    INTERSECT
    SELECT v FROM A WHERE fs = c.fs
  );
""")
res


┌───────┬───────┬───────┐
│  Id1  │  Id2  │  Id3  │
│ int32 │ int32 │ int32 │
├───────┼───────┼───────┤
│    99 │   252 │   331 │
│    99 │   252 │   252 │
│    99 │   252 │   245 │
│    92 │   245 │   314 │
│    89 │   245 │   314 │
│    94 │   139 │   136 │
│    88 │   138 │   138 │
│    87 │   138 │   138 │
│    57 │   138 │   138 │
│   138 │   345 │   343 │
│     · │    ·  │    ·  │
│     · │    ·  │    ·  │
│     · │    ·  │    ·  │
│    94 │   343 │   344 │
│   314 │   344 │   345 │
│   252 │   344 │   345 │
│   139 │   344 │   345 │
│    94 │   344 │   345 │
│    99 │   345 │   346 │
│    89 │   345 │   346 │
│    57 │   345 │   346 │
│    92 │   314 │   113 │
│    92 │   346 │   345 │
├───────┴───────┴───────┤
│ 1307 rows (20 shown)  │
└───────────────────────┘

In [9]:
res = duckdb.sql("""
with A as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    limit 10_000
)
select distinct a.fs, b.fs, c.fs
from A as a
join A as b
on a.v = b.v and a.fs < b.fs
join A as c
on a.v = c.v and a.fs < c.fs
""")
res

┌───────┬───────┬───────┐
│  fs   │  fs   │  fs   │
│ int32 │ int32 │ int32 │
├───────┼───────┼───────┤
│    26 │   331 │    57 │
│    26 │   113 │    57 │
│    57 │   245 │    92 │
│    57 │   175 │    92 │
│    57 │   331 │   113 │
│    26 │   314 │    99 │
│    57 │   314 │    99 │
│    26 │   331 │   175 │
│    26 │   113 │   175 │
│    26 │   314 │   245 │
│     · │    ·  │    ·  │
│     · │    ·  │    ·  │
│     · │    ·  │    ·  │
│   252 │   314 │   314 │
│   113 │   252 │   314 │
│    99 │   113 │   331 │
│   175 │   331 │   314 │
│    26 │    92 │    57 │
│    26 │   331 │   314 │
│    26 │    92 │   175 │
│    26 │   113 │   314 │
│    26 │    57 │   113 │
│    94 │   140 │   139 │
├───────┴───────┴───────┤
│  356 rows (20 shown)  │
└───────────────────────┘

In [10]:
res = duckdb.sql("""
with A as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    limit 10_000
),
union
select distinct a.fs, b.fs, c.fs
from A as a
join A as b
on a.v = b.v and a.fs < b.fs
join A as c
on a.v = c.v and a.fs < c.fs
""")
res

NameError: name 'kaas' is not defined

## Search
Now, write a query to find tables that are already concatenated.

In [ ]:
res = duckdb.sql("""
with A as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    --where fs = 138384
),
B as (
    select * from '../experiments/results/parquet_tall/df_*.parquet'
    --where fs = 138390
),
moved_B as (
    select index, fs, 1 + i + ((select max(i) from A group by fs) - (select min(i) from B group by fs)) as i, j, v
    from B
)
select a.fs, b.fs from
    (select distinct fs from A) as a
        cross join
    (select distinct fs from A) as b
        cross join
    (select distinct fs from A) as c
        on A.fs < B.fs and A.fs < C.fs
        and exists(

        )
limit 1
""")
res.show()
